In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import bq_helper
from bq_helper import BigQueryHelper
# https://www.kaggle.com/sohier/introduction-to-the-bq-helper-package
google_analytics = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                   dataset_name="data:google_analytics_sample")

Using Kaggle's public dataset BigQuery integration.


In [3]:
bq_assistant = BigQueryHelper("bigquery-public-data", "google_analytics_sample")

Using Kaggle's public dataset BigQuery integration.


In [4]:
#query1 = """SELECT * FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20160801`;"""
#response1 = google_analytics.query_to_pandas_safe(query1)

In [5]:
#Unique pages visited in all sessions combined
query2 = """SELECT hits.page.pagePath
            FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20160802` AS GA, UNNEST(GA.hits) AS hits
            GROUP BY hits.page.pagePath"""

response2 = google_analytics.query_to_pandas_safe(query2)

response2

,pagePath
0,/home
1,/google+redesign/apparel/men++s
2,/google+redesign/bags
3,/google+redesign/drinkware
4,/google+redesign/electronics
...,...
244,/google+redesign/apparel/womens/womens+tshirts...
245,/google+redesign/apparel/women+s+google+perfor...
246,/google+redesign/apparel/women+s+google+short+...
247,/google+redesign/apparel/womens/womens+tshirts...


In [6]:
#number of pageviews
query3 = """SELECT hits.page.pagePath, count(*) as page_views
            FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20160802` AS GA, UNNEST(GA.hits) AS hits
            WHERE hits.type = 'PAGE'
            GROUP BY hits.page.pagePath
            ORDER BY page_views desc
            """
response3 = google_analytics.query_to_pandas(query3)
response3

,pagePath,page_views
0,/home,2664
1,/basket.html,830
2,/google+redesign/office,481
3,/google+redesign/bags,446
4,/google+redesign/electronics,436
...,...,...
244,/google+redesign/apparel/womens/womens+tshirts...,1
245,/google+redesign/apparel/women+s+google+perfor...,1
246,/google+redesign/apparel/women+s+google+short+...,1
247,/google+redesign/apparel/womens/womens+tshirts...,1


In [7]:
#Unique number of unique pageviews --> counts the pageviews by an user only once
query4 = """SELECT pagePath, COUNT(*) as page_views, COUNT(DISTINCT session_id) as unique_page_views
            FROM
            (SELECT hits.page.pagePath, CONCAT(fullVisitorId, CAST(visitStartTime as STRING)) as session_id
            FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20160802` AS GA, UNNEST(GA.hits) AS hits
            WHERE hits.type = 'PAGE')
            GROUP BY pagePath
            ORDER BY page_views desc
            """
response4 = google_analytics.query_to_pandas(query4)
response4

,pagePath,page_views,unique_page_views
0,/home,2664,1798
1,/basket.html,830,198
2,/google+redesign/office,481,342
3,/google+redesign/bags,446,320
4,/google+redesign/electronics,436,329
...,...,...,...
244,/google+redesign/apparel/womens/womens+tshirts...,1,1
245,/google+redesign/apparel/women+s+google+perfor...,1,1
246,/google+redesign/apparel/women+s+google+short+...,1,1
247,/google+redesign/apparel/womens/womens+tshirts...,1,1


In [8]:
#Exits
query5 = """SELECT pagePath, sum(exits) AS exits
            FROM
                (SELECT hits.page.pagePath, 
                CASE 
                    WHEN hits.isExit IS NOT NULL THEN 1
                    ELSE 0
                END AS exits
                FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20160802` AS GA, UNNEST(GA.hits) AS hits
                WHERE hits.type = 'PAGE')
            GROUP BY pagePath
            ORDER BY exits DESC
            """
response5 = google_analytics.query_to_pandas(query5)
response5

,pagePath,exits
0,/home,1022
1,/google+redesign/office,72
2,/google+redesign/apparel/men++s/men++s+t+shirts,71
3,/google+redesign/bags,67
4,/google+redesign/drinkware,57
...,...,...
244,/google+redesign/apparel/men+s+t+shirts/short+...,0
245,/google+redesign/apparel/womens/womens+tshirts...,0
246,/google+redesign/apparel/women+s+google+perfor...,0
247,/google+redesign/apparel/womens/womens+tshirts...,0


In [9]:
#Total Time on Page
